## BLE Heart Rate Streaming (Garmin) – Notebook Interface

### Cell 1 – Module Imports
These are the required modules:

- `bleak`: for Bluetooth Low Energy scanning and connection
- `asyncio`: to enable asynchronous BLE interactions
- `datetime`: for timestamping/logging heart rate values

In [ ]:
from bleak import BleakClient
from bleak import BleakScanner
import asyncio
from datetime import datetime

### Cell 2 – Scan 📶 for Available BLE Devices
This function uses `BleakScanner.discover()` to find nearby Bluetooth Low Energy devices. It prints the address and name of each discovered device.

In [ ]:
devices = await BleakScanner.discover()
for d in devices:
    print(d)

#### 📦 Sample Output:
```
D4:4A:C4:B0:24:03: Forerunner

This is called Media Access Control (MAC) - A hardware Identifier.
```

### Cell 3 – Connect 🔗 to Device and List Available Services
Replace the address below with your device's BLE MAC. This connects to the Garmin device and prints all available **services and characteristics**.

In [ ]:
address = "D4:4A:C4:B0:24:03"  # Your Garmin BLE MAC address

async def main():
    async with BleakClient(address) as client: # Create client instance/connection
        print("Connected:", client.is_connected)

        # Loop and print all services and characteristics with their description and properties in a hierarchy

        print("\nAvailable services:\n")
        for service in client.services:
            print(f"[Service] {service.uuid}: {service.description}")
            for char in service.characteristics:
                print(f"  ├─ [Char] {char.uuid} ({char.properties})")

await main()

#### 📦 Sample Output:
```
Connected: True

Available services:

[Service] 00001800-0000-1000-8000-00805f9b34fb: Generic Access Profile
  ├─ [Char] 00002a00-0000-1000-8000-00805f9b34fb (['read'])
  ├─ [Char] 00002a01-0000-1000-8000-00805f9b34fb (['read'])
  ├─ [Char] 00002a04-0000-1000-8000-00805f9b34fb (['read'])
  ├─ [Char] 00002aa6-0000-1000-8000-00805f9b34fb (['read'])
[Service] 00001801-0000-1000-8000-00805f9b34fb: Generic Attribute Profile
  ├─ [Char] 00002a05-0000-1000-8000-00805f9b34fb (['indicate'])
[Service] 6a4e8022-667b-11e3-949a-0800200c9a66: Unknown
  ├─ [Char] 6a4e4c80-667b-11e3-949a-0800200c9a66 (['write-without-response'])
  ├─ [Char] 6a4ecd28-667b-11e3-949a-0800200c9a66 (['notify', 'read'])
[Service] 0000180d-0000-1000-8000-00805f9b34fb: Heart Rate
  ├─ [Char] 00002a37-0000-1000-8000-00805f9b34fb (['notify'])
[Service] 00001814-0000-1000-8000-00805f9b34fb: Running Speed and Cadence
  ├─ [Char] 00002a54-0000-1000-8000-00805f9b34fb (['read'])
  ├─ [Char] 00002a53-0000-1000-8000-00805f9b34fb (['notify'])
...
```

### Cell 4 – Print ❤️ Heart Rate Notification 
Use you Device MAC and UUID for Heart Rate. This connects to the Heart Rate service and prints **Heart Rate in Beats Per Min**.

In [ ]:
# -----------------------------------------------
# BLE Configuration for Garmin Heart Rate Monitor
# -----------------------------------------------

# Device-specific MAC address
address = "D4:4A:C4:B0:24:03"  

# UUID for the Heart Rate Measurement Characteristic
# Used to subscribe to notifications from the device
HR_CHAR_UUID = "00002a37-0000-1000-8000-00805f9b34fb"


# ------------------------------------------------
# Callback Function: Processes heart rate data
# ------------------------------------------------
# → This function is passed to start_notify()
# → Triggered each time the device sends heart rate data
# → Parameters `sender` and `data` are passed by Bleak

def handle_hr_notification(sender, data):
    # Byte structure: [flags, heart_rate, ...]
    heart_rate = data[1]  # ← Byte 1 holds the HR value
    print(f"Heart Rate: {heart_rate} bpm")


# --------------------------------------------------------------
# Main async function: Connects, subscribes, listens for HR Data
# --------------------------------------------------------------
async def main():
    # Connect to device using MAC address
    async with BleakClient(address) as client:
        print("Connected:", client.is_connected)

        # Subscribe to notifications from HR characteristic
        # Uses: HR_CHAR_UUID + callback function
        await client.start_notify(HR_CHAR_UUID, handle_hr_notification)
        print("Listening for heart rate notifications...")

        await asyncio.sleep(10)  # Listen duration (in seconds)

        # Unsubscribe after listening
        await client.stop_notify(HR_CHAR_UUID)
        print("Stopped notifications.")


# ------------------------------------------------
# Entry point: Run the async main function
# ------------------------------------------------
await main()


#### 📦 Sample Output:

````
Connected: True
Listening for heart rate notifications...
Heart Rate: 82 bpm
Heart Rate: 82 bpm
Heart Rate: 85 bpm
````